In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import albumentations as A
from pathlib import Path
import os
from tqdm import tqdm
import sklearn.metrics
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib.font_manager import FontProperties
prop = FontProperties()
plt.style.use('seaborn-dark-palette')
prop.set_file('../data/kalpurush.ttf')
%matplotlib inline

In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from IPython.display import SVG
from tensorflow.keras import backend as K
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D

In [4]:
dataframe = pd.read_parquet('../data/train_labels.parquet')

In [21]:
dataframe = dataframe.sample(n=20084, random_state=42)

In [112]:
dataframe.columns


Index(['image_id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '4089', '4090', '4091', '4092', '4093', '4094', '4095', 'grapheme_root',
       'vowel_diacritic', 'consonant_diacritic'],
      dtype='object', length=4100)

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(dataframe.drop(['image_id','grapheme_root','vowel_diacritic', 'consonant_diacritic'],axis=1))

---

## Generators

In [24]:
from sklearn.model_selection import train_test_split

train_df_1, valid_df_1 = train_test_split(dataframe.drop('image_id',axis=1), random_state=42,
                                          stratify=dataframe[['grapheme_root',
                                                                    'vowel_diacritic',
                                                                    'consonant_diacritic']])

In [67]:
x_train = train_df_1.drop(['grapheme_root','vowel_diacritic','consonant_diacritic'],axis=1)

In [68]:
y_train = train_df_1[['grapheme_root','vowel_diacritic','consonant_diacritic']]

In [69]:
x_valid = valid_df_1.drop(['grapheme_root','vowel_diacritic','consonant_diacritic'],axis=1)

In [70]:
y_valid = valid_df_1[['grapheme_root','vowel_diacritic','consonant_diacritic']]

In [71]:
x_train = x_train.values.reshape(15063,64,64,1)

In [72]:
def image_scale(image):
    augmentation_pipeline = A.Resize(224,224,always_apply=True)
    images_aug = augmentation_pipeline(image = image)['image']
    
    return images_aug

In [78]:
train_im_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_im_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [79]:
train_im_generator.fit(x_train)

In [87]:
def generate_data_generator(generator, X, Y1):
    genX = generator.flow(X, seed=42)
    genY1 = generator.flow(Y1, seed=42)
    while True:
            Xi = genX.next()
            Yi = genY1.next()
            Yi0 = Yi[0]
            Yi1 = Yi[1]
            Yi2 = Yi[2]
            yield Xi, [Yi0, Yi1, Yi2]

In [109]:
data_gen = generate_data_generator(train_im_generator,x_train,y_train)

In [89]:
def model_alexnet():
    
    input_ = Input(shape=(64,64,1))
    
    conv1 = Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', activation='relu')(input_)
    maxpool1 = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')(conv1)
    bn1 = BatchNormalization()(maxpool1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='same', activation='relu')(bn1)
    maxpool2 = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')(conv2)
    bn2 = BatchNormalization()(maxpool2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(bn2)
    drop1 = Dropout(0.5)(conv3)
    conv4 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(drop1)
    drop2 = Dropout(0.5)(conv4)
    conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(drop2)
    
    flat = Flatten()(conv5)
    
    dense_root_1 = Dense(4096,activation='relu')(flat)
    dense_root_2 = Dense(4096,activation='relu')(dense_root_1)
    dense_root_3 = Dense(1000,activation='relu')(dense_root_2)
    output_root = Dense(168,activation='softmax',name='root')(dense_root_3)
    
    dense_vowel_1 = Dense(800,activation='relu')(flat)
    dense_vowel_2 = Dense(600,activation='relu')(dense_vowel_1)
    dense_vowel_3 = Dense(100,activation='relu')(dense_vowel_2)
    output_vowel = Dense(11,activation='softmax',name='vowel')(dense_vowel_3)
    
    dense_consonant_1 = Dense(800,activation='relu')(flat)
    dense_consonant_2 = Dense(600,activation='relu')(dense_consonant_1)
    dense_consonant_3 = Dense(100,activation='relu')(dense_consonant_2)
    output_consonant = Dense(7,activation='softmax',name='consonant')(dense_consonant_3)
    
    model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])
    
    return model

In [90]:
keras.backend.clear_session()
model_alexnet = model_alexnet()
#keras.utils.plot_model(model_alexnet, '../results/alexnet.png', expand_nested=True, show_shapes=True);

In [91]:
model_alexnet.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

In [103]:
type(x_train)

numpy.ndarray

In [108]:
history_alexnet = model_alexnet.fit(x_train,
                                    (y_train['grapheme_root'].values, y_train['vowel_diacritic'].values,
                                     y_train['consonant_diacritic'].values), batch_size=1, epochs=1)

Train on 15063 samples
  144/15063 [..............................] - ETA: 24:11 - loss: 5927108.1707 - root_loss: 5889568.0000 - vowel_loss: 34454.9844 - consonant_loss: 3082.9541 - root_accuracy: 0.0280 - vowel_accuracy: 0.1189 - consonant_accuracy: 0.6014

KeyboardInterrupt: 